In [1]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
from evaluate_data import EXP_NAME, RES_DIRNAME, FS, DURATION_SEC
from evaluate_data import read_df, add_soundlevel, read_df_from_wav
from frequency_analysis import get_spectrogram

def custom_save(function=plt.savefig, fname=""):
    if not os.path.exists(os.path.dirname(fname)):
        os.makedirs(os.path.dirname(fname))
        print('created new directory', os.path.dirname(fname))
    try:
        # to make sure suptitle is not cut off
        function(fname, bbox_inches="tight")
    except:
        function(fname)
    print("saved as", fname)

def plot_spectrogram(spectrogram, label):
    plt.figure()
    plt.pcolormesh(np.log(spectrogram))
    plt.xlabel('time index [-]')
    plt.ylabel('frequency index [-]')
    plt.title(label)
    
def plot_soundlevel(df, label, ax=None):
    if ax is None:
        fig, ax = plt.subplots()
    ax.semilogy(df.timestamp_s, df.sound_level, label=label, marker="o")
    ax.set_xlabel('time [s]')
    ax.set_ylabel('sound level')

## Compare sound levels

### combined

In [ ]:
#duration = 10 * 1e3 # miliseconds from end
duration = DURATION_SEC * 1e3 # miliseconds from end

exp_name = "2020_09_17_white-noise-static"
#exp_name = "2020_10_14_static"
assert exp_name == EXP_NAME, "watch out, possible discrepancy between evaluate_data.py and here"

source = "random_linear"
df_source, pose_source  = read_df(exp_name=exp_name,
    degree=0, props=False, snr=False, motors=False, source=source)
df_all, pose_all = read_df(exp_name=exp_name,
    degree=0, props=False, snr=False, motors=True, source=source)
df_props, pose_props = read_df(exp_name=exp_name,
    degree=0, props=False, snr=False, motors=True, source=None)

spectrogram_source, __ = get_spectrogram(df_source)
spectrogram_all, __ = get_spectrogram(df_all)
spectrogram_props, __ = get_spectrogram(df_props)

plot_spectrogram(spectrogram_source, label="source")
plot_spectrogram(spectrogram_all, label="all")
plot_spectrogram(spectrogram_props, label="props")

add_soundlevel(df_source, duration=duration)
add_soundlevel(df_all, duration=duration)
add_soundlevel(df_props, duration=duration)

fig, ax = plt.subplots()
plot_soundlevel(df_source, "source", ax=ax)
plot_soundlevel(df_all, "all", ax=ax)
plot_soundlevel(df_props, "props", ax=ax)
plt.legend()
plt.title('audio deck signals, selected 32 frequencies')

### individual

In [ ]:
sound_levels = np.abs(np.array([*df_all.loc[:, "signals_f"]]))
print(sound_levels.shape)

skip = 5
n_times = sound_levels.shape[0]
n_mics = sound_levels.shape[1]
n_freqs = sound_levels.shape[2]

times = df_all.timestamp_s.values
frequencies = df_all.iloc[0].frequencies

fig, axs = plt.subplots(n_mics, sharex=True)
fig.set_size_inches(10,10)
for mic_i in range(n_mics):
    axs[mic_i].set_title(f'mic{mic_i}')
    for j in range(n_freqs)[::skip]:
        axs[mic_i].plot(times, sound_levels[:, mic_i, j], label=f"{frequencies[j]}Hz")
    axs[mic_i].plot(times, np.median(sound_levels[:, mic_i, :], axis=-1), label="mean", color='black', linewidth=2.0)
[ax.set_yscale('log') for ax in axs]
axs[0].legend(loc="upper left", bbox_to_anchor=[1.0, 1.0])

In [ ]:
if exp_name == "2020_09_17_white-noise-static":
    fname_source = f'../experiments/{exp_name}/export/crazyflie_audio_measurements_nomotors_nosnr_noprops_source_45.wav'
    fname_all =    f'../experiments/{exp_name}/export/crazyflie_audio_measurements_motors_nosnr_noprops_source_45.wav'
    fname_props =  f'../experiments/{exp_name}/export/crazyflie_audio_measurements_motors_nosnr_noprops_nosource_45.wav'
    thresh = 1e2
else:
    fname_source = f'../experiments/{exp_name}/export/nomotors_nosnr_noprops_{source}.wav'
    fname_all =    f'../experiments/{exp_name}/export/motors_nosnr_noprops_{source}.wav'
    fname_props =  f'../experiments/{exp_name}/export/motors_nosnr_noprops_None.wav'
    thresh = 1e-10
print('reading', fname_source)

df_wav_all = read_df_from_wav(fname_all) 
#df_wav_source = read_df_from_wav(fname_source) 
#df_wav_props = read_df_from_wav(fname_props) 
#df_wav_all.to_pickle(fname_all.replace('.wav', '.pk'))
#df_wav_source.to_pickle(fname_source.replace('.wav', '.pk'))
#df_wav_props.to_pickle(fname_props.replace('.wav', '.pk'))

df_wav_all = pd.read_pickle(fname_all.replace('.wav', '.pk'))
df_wav_source = pd.read_pickle(fname_source.replace('.wav', '.pk'))
df_wav_props = pd.read_pickle(fname_props.replace('.wav', '.pk'))

add_soundlevel(df_wav_source, threshold=thresh)
add_soundlevel(df_wav_all, threshold=thresh)
add_soundlevel(df_wav_props, threshold=thresh)

fig, ax = plt.subplots()
plot_soundlevel(df_wav_source, label="source", ax=ax)
plot_soundlevel(df_wav_all, label="all", ax=ax)
plot_soundlevel(df_wav_props, label="props", ax=ax)
plt.legend()
plt.title('measurement mic signals, all 512 frequencies')

In [ ]:
def select_bins(row):
    signals_f = row.signals_f[:, bins_wav_selected]
    frequencies = row.frequencies[bins_wav_selected]
    row.frequencies = frequencies
    row.n_frequencies = len(frequencies)
    return row

freqs_selected = df_source.iloc[0].loc['frequencies']
freqs_wav = df_wav_source.iloc[0].loc['frequencies']
bins_wav_selected = [np.argmin(np.abs(freq - freqs_wav)) for freq in freqs_selected]
print('selected bins:', bins_wav_selected)

df_wav_selected_all = df_wav_all.apply(select_bins, axis=1)
df_wav_selected_source = df_wav_source.apply(select_bins, axis=1)
df_wav_selected_props = df_wav_props.apply(select_bins, axis=1)

print(f"reduced from {len(df_wav_all.loc[0, 'frequencies'])} to {len(df_wav_selected_all.loc[0, 'frequencies'])}")

add_soundlevel(df_wav_selected_source, duration=duration, threshold=thresh)
add_soundlevel(df_wav_selected_all, duration=duration, threshold=thresh)
add_soundlevel(df_wav_selected_props, duration=duration, threshold=thresh)

fig, ax = plt.subplots()
plot_soundlevel(df_wav_selected_source, label="source", ax=ax)
plot_soundlevel(df_wav_selected_all, label="all", ax=ax)
plot_soundlevel(df_wav_selected_props, label="props", ax=ax)
plt.legend()
plt.title('measurement mic signals, selected 32 frequencies')

## Plot results

In [ ]:
import itertools
from copy import deepcopy
from evaluate_data import degree_list, method_list, combine_list, normalize_list

def filter_df_by_dict(result_df, this_dict, verbose=False):
    this_df = result_df.copy()
    for key, val in this_dict.items():
        if not key in this_df.columns:
            continue
            
        if not len(this_df.loc[this_df[key]==val]):
            if verbose:
                print(f'did not find any {key}=={val} in results. Available: {this_df[key].unique()}')
        this_df = this_df.loc[this_df[key]==val]
        if not len(this_df): # can stop filtering, already empty
            break
    return this_df

def plot_spectrum(full_spectrum, title='', ax=None, log=False, eps=1e-30):
    if ax is None:
        fig, ax = plt.subplots()
    else:
        fig = plt.gcf()
        
    fig.set_size_inches(10, 5)
    if log:
        full_spectrum = deepcopy(full_spectrum)
        full_spectrum[full_spectrum <= eps] = np.nan
        ax.pcolormesh(range(full_spectrum.shape[1]), angles, np.log(full_spectrum))
    else:
        ax.pcolormesh(range(full_spectrum.shape[1]), angles, full_spectrum)
    ax.set_xlabel('time idx [-]')
    ax.set_ylabel('angle [deg]')
    ax.set_title(title)

fname = f'{RES_DIRNAME}/static_spectra.pkl'
try:
    result_df = pd.read_pickle(fname)
except FileNotFoundError: 
    print(f'Did not find {fname}. Did you run evaluate_data.py? ')

## A. Ideal case: what is best combination of method+combine+normalize? 

Most combinations work fine, this is probably not an important factor.

In [ ]:
from audio_stack.beam_former import normalize_rows

base_dict = dict(
    motors=False,
    props=False,
    snr=False, 
    degree=0, 
    source='mono_linear',
    #source='random_linear',
    #source='mono_linear',
    # will be set later
    method=None,
    combine=None,
    normalize=None,
)

normalize_colors = {norm: f"C{i}" for i, norm in enumerate(normalize_list)}
#combine_ls = {"product": ":", "sum": "-"}
method_ls = {"mvdr": ":", "das": "-"}

fig_tot, ax_list = plt.subplots(1, len(combine_list), sharex=True, sharey=False)
fig_tot.set_size_inches(10, 5)
ax_dict = dict(zip(combine_list, ax_list))
[ax.set_title(title) for title, ax in ax_dict.items()]

for method, normalize, combine in itertools.product(method_list, normalize_list, combine_list):
    this_dict = base_dict.copy()
    this_dict["method"] = method
    this_dict["combine"] = combine
    this_dict["normalize"] = normalize
    this_df = filter_df_by_dict(result_df, this_dict)
            
    if not len(this_df):
        continue 
        
    gt_degree = 180 - this_dict["degree"]
        
    # angles x times
    full_spectrum = np.array([*this_df.loc[:, "spectrum"]]).T
    title = f"method:{method}, normalize:{normalize}, combine:{combine}"
    
    full_spectrum = normalize_rows(full_spectrum.T, method="zero_to_one_all").T
    
    angles = np.linspace(0, 360, full_spectrum.shape[0])
    full_spectrum = normalize_rows(full_spectrum.T, method="sum_to_one").T
    
    fig, ax = plt.subplots()
    plot_spectrum(full_spectrum, title=title, ax=ax, log=True)
    ax.axhline(gt_degree, color='orange', ls=':')
    
    mean_spectrum = np.mean(full_spectrum, axis=1)
    label = f"{str.upper(method)}, {normalize}"
    ax_dict[combine].semilogy(angles, mean_spectrum, color=normalize_colors[normalize], 
                ls=method_ls[method], label=label)
ax_list[1].legend(loc="upper left", bbox_to_anchor=[1.0, 1.0])
[ax.set_xlabel("angle [deg]") for ax in ax_list]

## B. no motors: does it help to use high-SNR bins? 

It doesn't help.

In [ ]:
base_dict = dict(
    motors=False,
    props=False,
    source='random_linear',
    #source='mono_linear',
    # fill from above 
    method='mvdr',
    combine='sum',
    normalize='zero_to_one',
    snr=None, 
    degree=None, # will be set later
)

for degree, snr in itertools.product(degree_list, [False, True]):
    gt_degree = 180 - degree
    
    this_dict = base_dict.copy()
    this_dict["degree"] = degree
    this_dict["snr"] = snr 
    this_df = filter_df_by_dict(result_df, this_dict)
            
    if not len(this_df):
        continue 
        
    # angles x times
    full_spectrum = np.array([*this_df.loc[:, "spectrum"]]).T
    title = f"{degree} deg, snr:{snr}"
    
    fig, ax = plt.subplots()
    plot_spectrum(full_spectrum, title=title, ax=ax)
    ax.axhline(gt_degree, color='orange', ls=':')

## C. motors: what filtering helps? 

Findings:
- SNR filtering throws off results.
- propeller filtering doesn't hurt, but also doesn't seem to help significantly. It might help a little bit to get better DOA estimates at low SNR. 

TODO: quantify the second hypothesis

In [ ]:
import matplotlib.gridspec as gridspec
from audio_stack.beam_former import combine_rows
    
base_dict = dict(
    motors=True,
    method='mvdr',
    combine='sum',
    source='random_linear',
    #source='mono_linear',
    normalize='zero_to_one',
    degree=None, 
    snr=None, 
    props=None, # will be filled later
)

fig_tot, ax_list = plt.subplots(len(degree_list), 2)
ax_list = ax_list.reshape((len(degree_list), 2))
fig_tot.set_size_inches(10, 10)
ax_dict_time = dict(zip(degree_list, ax_list[:, 0]))
ax_dict_cdf = dict(zip(degree_list, ax_list[:, 1]))

for degree, snr, props in itertools.product(degree_list, [False, True], [False, True]):
    gt_degree = 180 - degree
    this_dict = base_dict.copy()
    this_dict["degree"] = degree
    this_dict["snr"] = snr 
    this_dict["props"] = props
    this_df = filter_df_by_dict(result_df, this_dict)
            
    if not len(this_df):
        continue 
        
    # angles x times
    full_spectrum = np.array([*this_df.loc[:, "spectrum"]]).T
    title = f"{degree} deg, snr:{snr}, props:{props}"
    angles = np.linspace(0, 360, full_spectrum.shape[0])
    
    fig = plt.figure(constrained_layout=True)
    spec = gridspec.GridSpec(ncols=3, nrows=1, figure=fig, width_ratios=[3, 1, 1])
    ax0 = fig.add_subplot(spec[0, 0])
    ax1 = fig.add_subplot(spec[0, 1], sharey=ax0)
    ax2 = fig.add_subplot(spec[0, 2], sharey=ax0)
    
    plot_spectrum(full_spectrum, title=title, ax=ax0)
    ax0.axhline(gt_degree, color='orange', ls=':')
    
    ax1.plot(combine_rows(full_spectrum.T, "sum"), angles)
    ax1.axhline(gt_degree, color='orange', ls=':')
    ax1.set_title('sum')
    
    ax2.plot(combine_rows(full_spectrum.T, "product"), angles)
    ax2.axhline(gt_degree, color='orange', ls=':')
    ax2.set_title('product')
    #axs[1].invert_xaxis()
    
    label = f"snr:{snr}, props:{props}"
    angle_errors = angles[np.argmax(full_spectrum, axis=0)]
    ax_dict_time[degree].scatter(range(full_spectrum.shape[1]), angle_errors, label=label)
    ax_dict_time[degree].axhline(gt_degree, color='black')
    ax_dict_time[degree].set_ylabel('angle [deg]')
    
    sorted_errors = sorted(np.abs(angle_errors - gt_degree))
    probs = np.linspace(0, 1, len(sorted_errors))
    ax_dict_cdf[degree].plot(sorted_errors, probs, label=label)
    ax_dict_cdf[degree].legend(loc='lower right')
    
ax_dict_time[degree].set_xlabel('time idx [-]')
ax_dict_cdf[degree].set_xlabel('abs angle error [deg]')

## D. Understand why SNR selection is not helping

Findings: 
- Looks like SNR selection doesn't help because most high-SNR are at low frequencies and potentially come from popeller noise. We need to either put up the freq_min to above 1500 degree or similar, or increase delta_freq for propeller filtering. 

In [ ]:
base_dict = dict(
    motors=True,
    method='mvdr',
    combine='sum',
    normalize='zero_to_one',
    #source='random_linear',
    source='mono_linear',
    degree=0, 
    props=True, 
    snr=None, # will be filled later
)

time_indices = range(90, 100)
n_times = len(time_indices)

min_freq = 1500

for snr, in itertools.product([False, True]):
    this_dict = base_dict.copy()
    this_dict["snr"] = snr 
    this_df = filter_df_by_dict(result_df, this_dict, verbose=False)
    
    if not len(this_df):
        continue 
        
    raw_spectrum = np.array([*this_df.loc[:, "spectrum_raw"]]).T
    raw_spectrum = raw_spectrum.transpose(0, 2, 1) # angles x times x frequencies
    
    gt_degree = 180 - this_dict["degree"]
    angles = np.linspace(0, 360, raw_spectrum.shape[0])
    
    fig, axs = plt.subplots(1, n_times+2, sharey=True)
    fig.suptitle(f"snr:{snr}")
    fig.set_size_inches(10, 5)
    for i in range(n_times):
        frequencies = this_df.iloc[time_indices[i]].frequencies
        
        frequencies_chosen = frequencies[frequencies > min_freq]
        #print('frequencies_chosen:', frequencies_chosen)
        
        raw_spectrum_flat = raw_spectrum[:, time_indices[i], frequencies > min_freq]
        axs[i].pcolormesh(range(len(frequencies_chosen)), angles, raw_spectrum_flat)
        axs[i].set_title(f"time {time_indices[i]}")
        axs[i].axhline(gt_degree, color='orange', ls=':')
        
    raw_spectrum_sum = np.sum(raw_spectrum[:, -n_times:, :].reshape(raw_spectrum.shape[0],-1), axis=1)
    raw_spectrum_product = np.sum(np.log10(raw_spectrum[:, -n_times:, :]).reshape(raw_spectrum.shape[0],-1), axis=1)
    axs[-2].plot(raw_spectrum_sum, angles)
    axs[-1].plot(raw_spectrum_product, angles)
    axs[-2].set_title('sum')
    axs[-1].set_title('product')

In [ ]:
# should pick propeller signals:
source = 'mono_linear'
df_snr_noprops,_ = read_df(degree=0, props=False, snr=True, motors=True, source=source)

# should pick between propellers (but maybe still too close?)
df_snr_props,_ = read_df(degree=0, props=True, snr=True, motors=True, source=source)

# should pick uniformly
df_nosnr_noprops,_ = read_df(degree=0, props=False, snr=False, motors=True, source=source)

if exp_name == '2020_09_17_white-noise-static':
    fname_props =  f'../experiments/{exp_name}/export/crazyflie_audio_measurements_motors_nosnr_noprops_nosource_45.wav'
else:
    fname_props =  f'../experiments/{exp_name}/export/motors_nosnr_noprops_None.wav'

In [ ]:
time = 0
fig, ax = plt.subplots()
for df, label in zip([df_snr_noprops, df_snr_props, df_nosnr_noprops], ["props", "between", "uniform"]):
    normalized_spec = np.sum(np.abs(df.iloc[time].signals_f), axis=0)
    normalized_spec -= np.min(normalized_spec)
    normalized_spec /= np.max(normalized_spec)
    ax.scatter(df.iloc[time].frequencies, normalized_spec, label=label)
    
normalized_spec = np.sum(np.abs(df_props.iloc[0].signals_f), axis=0)
normalized_spec -= np.min(normalized_spec)
normalized_spec /= np.max(normalized_spec)
ax.plot(df_props.iloc[0].frequencies, normalized_spec, color="C3", label='measurement mics')
#ax.set_yscale('log')
ax.legend()
ax.set_xlim([0, 4000])
fig.set_size_inches(10, 5)

## F. Average over time

Take the average of a few samples over time, considering the drone's movement. Plot the angle estimate vs. ground truth over time. 

In [ ]:
from audio_stack.beam_former import BeamFormer
from audio_stack.beam_former import normalize_rows

base_dict = dict(
    motors=True,
    method='mvdr',
    combine='sum',
    normalize='zero_to_one',
    #source = 'random_linear',
    source = True,
    degree=0, # 20 does not exist
    props=False, 
    snr=False
)

n_times = 4
combination_n = 3
combination_method = "sum"
normalization_method = "none"

this_df = filter_df_by_dict(result_df, base_dict, verbose=True)

beam_former = BeamFormer()
beam_former.init_dynamic_estimate(combination_n, combination_method, normalization_method)
angles = beam_former.theta_scan

combined_heatmap = None
raw_heatmap = None

for i, row in this_df.iterrows():
    raw_spectrum = row.spectrum_raw # frequencies x angles
    
    # TODO(FD): read orientation from current pose estimate.
    beam_former.add_to_dynamic_estimates(raw_spectrum, orientation_deg=0)
    estimate = beam_former.get_dynamic_estimate()
    
    combined_estimate = combine_rows(estimate, combination_method, keepdims=True)
    combined_estimate = normalize_rows(combined_estimate, method="zero_to_one")
    
    # TODO(FD): could also read this directly from df. Refactor? 
    raw_estimate = combine_rows(raw_spectrum, combination_method, keepdims=True)
    raw_estimate = normalize_rows(raw_estimate, method="zero_to_one")
    
    if combined_heatmap is None:
        combined_heatmap = combined_estimate.flatten()
    else:
        combined_heatmap = np.c_[combined_heatmap, combined_estimate.flatten()]
        
    if raw_heatmap is None:
        raw_heatmap = raw_estimate.flatten()
    else:
        raw_heatmap = np.c_[raw_heatmap, raw_estimate.flatten()]
    
fig, ax = plt.subplots()
ax.pcolormesh(range(combined_heatmap.shape[1]), angles, combined_heatmap)
ax.set_title('combined heatmap')

fig, ax = plt.subplots()
ax.pcolormesh(range(raw_heatmap.shape[1]), angles, raw_heatmap)
ax.set_title('raw heatmap')

In [ ]:
## plot angle estimate over time

# G. Evaluate frequency bins

plot which frequency bins yield the correct angle estimate, to see which frequency range
we should be using

In [ ]:
from audio_stack.beam_former import BeamFormer
from audio_stack.beam_former import normalize_rows

base_dict = dict(
    motors=True,
    method='mvdr',
    combine='sum',
    normalize='zero_to_one',
    source=True, #'random_linear',
    degree=None, # 20 does not exist
    props=False, 
    snr=False,
)
n_frequencies = 32
min_time_idx = 80 
counter = {j:[] for j in range(n_frequencies)}

heatmap = np.zeros((n_frequencies, len(angles)))

for degree in [0]:
    gt_degree = 180 - degree
    print(gt_degree)
    
    this_dict = base_dict.copy()
    this_dict["degree"] = degree
    
    this_df = filter_df_by_dict(result_df, this_dict, verbose=False)
    frequencies = this_df.iloc[0].frequencies

    for i, row in this_df.iloc[min_time_idx:].iterrows():
        raw_spectrum = row.spectrum_raw # frequencies x angles

        # for each frequency bin, find out the angle estimate.
        for j in range(n_frequencies):
            angle_idx = np.argmax(raw_spectrum[j, :])
            angle_error = abs(angles[angle_idx] - gt_degree)
            counter[j].append(angle_error)
            
            heatmap[j, angle_idx] += 1

fig, ax = plt.subplots()
for freq, error_list in counter.items():
    ax.scatter([frequencies[freq]]*len(error_list), error_list, alpha=0.1)
ax.set_xlabel('frequency')
ax.set_ylabel('counter')
ax.set_title('angle error')

fig, ax = plt.subplots()
ax.pcolormesh(frequencies, angles, heatmap.T)
ax.set_xlabel('frequency')
ax.set_ylabel('angle [deg]')
ax.set_title('counter of angle estimates')